In [1]:
# import sys
# !{sys.executable} -m pip install openpyxl
import pandas as pd
import json

In [2]:
res_oly = pd.read_excel("raw/paris_olympics_results.xlsx")
res_sib = pd.read_excel("raw/sibling_olympics_results.xlsx")
events = pd.read_excel("raw/olympic_events.xlsx")

In [3]:
res_oly

,Event,Sport,Lane,Athlete,Country,TimeSeconds
0,"Men's 10,000m",Athletics,1,Yann Schrub,FRA,NaN
1,"Men's 10,000m",Athletics,2,Benard Kibet,KEN,1603.98
2,"Men's 10,000m",Athletics,3,Célestin Ndikumana,BDI,NaN
3,"Men's 10,000m",Athletics,4,Tomoki Ota,JPN,1752.48
4,"Men's 10,000m",Athletics,5,Jacob Kiplimo,UGA,1606.39
...,...,...,...,...,...,...
460,Women's 800m Freestyle,Swimming,4,Katie Ledecky,USA,491.04
461,Women's 800m Freestyle,Swimming,5,Paige Madden,USA,493.00
462,Women's 800m Freestyle,Swimming,6,Lani Pallister,AUS,501.09
463,Women's 800m Freestyle,Swimming,7,Simona Quadarella,ITA,494.55


In [4]:
def combine_results(res_oly, res_sib):
    oly_max_lanes = res_oly.groupby(['Event', 'Sport'], as_index=False)['Lane'].max()
    res_sib = res_sib.merge(oly_max_lanes, on=['Event', 'Sport'], how='left', suffixes=('', '_max'))
    res_sib['Lane'] = res_sib.groupby(['Event', 'Sport']).cumcount() + res_sib['Lane_max'] + 1
    res_sib.drop(columns=['Lane_max'], inplace=True)
    res_combined = pd.concat([res_oly, res_sib], ignore_index=True).sort_values(by = ['Sport', 'Event', 'Lane'])
    res_combined = res_combined.loc[~res_combined['Lane'].isnull()]
    res_combined['Lane'] = res_combined['Lane'].astype(int)
    res_combined.loc[res_combined['TimeSeconds'].isnull(), 'TimeSeconds'] = 1e6
    return res_combined

res = combine_results(res_oly, res_sib)

In [5]:
def create_event_json(events_df, results_df, output_file):

    
    # Prepare a list to hold event data
    events_list = []

    # Iterate through each event in the events DataFrame
    for _, event_row in events_df.iterrows():
        event_name = event_row['Event']
        
        # Extract event details
        event_details = {
            "event": event_name,
            "sport": event_row['Sport'],
            "distance_m": event_row['DistanceM'],
            "laps": event_row['Laps'],
            "results": []
        }
        
        # Filter results for this event
        event_results = results_df[results_df['Event'] == event_name]

        # Iterate through each result row and add to event results
        for _, result_row in event_results.iterrows():
            result_details = {
                "lane": result_row['Lane'],
                "athlete": result_row['Athlete'],
                "country": result_row['Country'],
                "timeSeconds": result_row['TimeSeconds']
            }
            event_details['results'].append(result_details)
        
        # Add the event to the list
        events_list.append(event_details)

    # Convert to JSON format
    output_data = {"events": events_list}
    
    # Write to JSON file
    with open(output_file, 'w') as json_file:
        json.dump(output_data, json_file, indent=4)

create_event_json(events, res, 'events.json')

In [ ]:
res.head()